In [1]:
# Required Python packages
from datetime import date, datetime, timedelta
import calendar
import pandas as pd
import numpy as np
import itertools
from numpy.linalg import multi_dot
from scipy.stats import norm
from scipy.stats import norm
from scipy.stats import bernoulli
import matplotlib.pyplot as plt
import math
import copy


In [1]:
########################
# Date based Functions #
########################

# All User-defined Functions

#The following function finds the date corresponding to the last thursday of a month in a year
#Input: Year, month - both integers
#Output: Python date of last thursday of the month
def find_last_thurs_date_of_month(year, month):
    calendar_of_the_month = calendar.monthcalendar(year, month)
    no_of_weeks = len(calendar_of_the_month)
    if (calendar_of_the_month[no_of_weeks-1][3] != 0):
        return date(year, month, calendar_of_the_month[no_of_weeks-1][3])
    else:
        return date(year, month, calendar_of_the_month[no_of_weeks-2][3])
    
# #The following function finds the date corresponding to the previous thursdays
# #Input: Date - Python date format, No.of. thursdays back
# #Output: The interested thursday python date
def date_of_prev_thurs(date, no_of_thurs_back):
    if (date.weekday() > 3):
        minus_days = date.weekday() - 3
        prev_thurs = date - timedelta(days=minus_days) -timedelta(days = 7 * (no_of_thurs_back-1))
        return prev_thurs
    else:
        prev_thurs = date - timedelta(days=no_of_thurs_back * 7)
        add_days = 3-prev_thurs.weekday()
        return (prev_thurs + timedelta(days=add_days))

# # #Input: Date - Python date format, No.of. thursdays back
# # #Output: The interested thursday python date
def date_of_next_thurs(date):
    if (date.weekday() > 3):
        minus_days = date.weekday() - 3
        next_thurs = date - timedelta(days=minus_days) + timedelta(days = 7)
        return next_thurs
    else:
        add_days = 3-date.weekday()
        return (date + timedelta(days=add_days) + timedelta(days = 7))    

# #Function for getting previous weekday of a date
# #Input: any python date
# #Output: Previous weekday python date
def prev_weekday(date):
    if (date.weekday() == 0):
        return date - timedelta(days=3)
    elif (date.weekday() == 6):
        return date - timedelta(days=2)
    else:
        return date - timedelta(days=1)

# #Function to Check for holidays- if holiday, the closing/expiry day is the previous working_day
# #if holiday, select prev working day
# #Input: python date, holiday python list
# #Output: Previous working python date
def prev_workday_if_holiday(date, holidays_list):
    if (date in holidays_list):
        return prev_workday_if_holiday(prev_weekday(date), holidays_list)
    else:
        return date

# #Check for holidays- if holiday, the closing day is the previous working_day
# # Following function is used to generate flow in 'For loops', 
# #if holiday, select next working day
def next_working_day_incl_input_date(date, holidays_list):
    if (date in holidays_list):
        return next_working_day_incl_input_date(next_weekday(date), holidays_list)
    else:
        return date

# #Function for getting next week days of a date
# #Input: any python date
# #Output: Next weekday python date
#0-Mon, 1-Tues, 2-Wed, 3-Thurs, 4-Fri, 5-Sat, 6-Sun
def next_weekday(date):
    if (date.weekday() == 4):
        return date + timedelta(days=3)
    elif (date.weekday() == 5):
        return date + timedelta(days=2)
    else:
        return date + timedelta(days=1)

# #Function for getting next weekday if date is not a weekday
# #Input: any python date
# #Output: Next weekday including inputted python date
def next_weekday_incl_input_date(date):
    if (date.weekday() == 5):
        return date + timedelta(days=2)
    elif (date.weekday() == 6):
        return date + timedelta(days=1)
    else:
        return date 
    
# # The function helps in finding the list option last thursday expiries of the month 
# #(we are interest to hedge options of this expiry)
# # Input: Dictionary with keys as years and each key has value of list of months we are interested
# # Output: List of monthly last thursday expiries
def get_all_monthly_option_expiries(periods_of_interest, holidays_list):
    monthly_expiries = []
    for key in periods_of_interest.keys():
        list_of_months = periods_of_interest[key]
        expiry_dates = [find_last_thurs_date_of_month(key, month) for month in list_of_months]
        expiry_dates = [prev_workday_if_holiday(date, holidays_list) for date in expiry_dates]
        monthly_expiries = monthly_expiries + expiry_dates
    return monthly_expiries

# #Function to find all weekly expiry option dates for every month of interest (i.e. every thursdays)
# #Input: List python monthly expiries
# #Output: dictionary - {'monthly_expiry_date:': [4weeks before thurs, 3w, 2w, 1w]}
# def find_wkly_expries(mthly_expiries, holidays_list):
#     weekly_expiries_dict = {}
#     for expiry_date in mthly_expiries:
#         wkly_expiries_list = [date_of_prev_thurs(expiry_date, no_of_thurs) for no_of_thurs in range(4, 0, -1)]
#         wkly_expiries_list = [prev_workday_if_holiday(date, holidays_list) for date in wkly_expiries_list]
#         weekly_expiries_dict[expiry_date.strftime("%d-%b-%Y")] = wkly_expiries_list
#     return weekly_expiries_dict

def find_wkly_expries(mthly_expiries, holidays_list):
    weekly_expiries_dict = {}
    for expiry_date in mthly_expiries:
        if (expiry_date.month == 1):
                last_month = 12
                last_year = expiry_date.year - 1
        else:
            last_month = expiry_date.month - 1
            last_year = expiry_date.year 
        
        last_thurs_of_prev_month = prev_workday_if_holiday(find_last_thurs_date_of_month(last_year, \
                                                                                         last_month), 
                                                           holidays_list)
        wkly_expiries_list = []
        date_of_interest = last_thurs_of_prev_month
        while(date_of_interest < expiry_date):
            wkly_expiries_list.append(date_of_interest)
            date_of_interest = prev_workday_if_holiday(date_of_next_thurs(date_of_interest), holidays_list)
        weekly_expiries_dict[expiry_date.strftime("%d-%b-%Y")] = wkly_expiries_list
    return weekly_expiries_dict


# Function to find next working day
def next_workday(date, holidays_list):
    if (next_weekday(date) in holidays_list):
        return next_workday(next_weekday(date), holidays_list)
    else:
        return next_weekday(date)    

# Function to generate all working days in a month (i.e. first thurs to fourth thursday - 1 month option life)
def generate_daily_dates_each_month(dict_wkly_expiries_each_month, holidays_list):
    dict_daily_dates = {}
    for each_month in dict_wkly_expiries_each_month:
        start_date = dict_wkly_expiries_each_month[each_month][0]
        end_date = datetime.strptime(each_month, "%d-%b-%Y").date()
        curr_date = start_date
        monthly_list = []
        while (curr_date < end_date):
            curr_date = next_workday(curr_date, holidays_list)
            monthly_list.append(curr_date)
        dict_daily_dates[each_month] = monthly_list
    return dict_daily_dates

def generate_daily_dates_each_month(dict_wkly_expiries_each_month, holidays_list):
    dict_daily_dates = {}
    for each_month in dict_wkly_expiries_each_month:
        start_date = dict_wkly_expiries_each_month[each_month][0]
        end_date = datetime.strptime(each_month, "%d-%b-%Y").date()
        curr_date = start_date
        monthly_list = [curr_date]
        while (curr_date < end_date):
            curr_date = next_workday(curr_date, holidays_list)
            monthly_list.append(curr_date)
        dict_daily_dates[each_month] = monthly_list
    return dict_daily_dates

# Find the week corresponding to a date
def find_week(date, holidays_list):
    prev_thurs = date_of_prev_thurs(date, 1)
    week = prev_workday_if_holiday(prev_thurs, holidays_list)
    return week

def create_df_wk_days(dict_daily_dates_error_cal, dict_wkly_expiries_each_month, holidays_list):
    day_wk ={}
    full_days_list = []
    full_week_list = []
    full_expiry_list = []
    full_month_list = []
    for each_month in dict_daily_dates_error_cal.keys():
        day_ind = 0
        week_ind = 0
        weeks = dict_wkly_expiries_each_month[each_month]
        no_of_wks = len(weeks)
        days = dict_daily_dates_error_cal[each_month]
        no_of_days = len(days)
        month = datetime.strptime(each_month, "%d-%b-%Y").date()
        days_list = []
        month_list = []
        week_list = []
        expiry_list = []
        while(day_ind < no_of_days):
            if (week_ind < no_of_wks - 2):
                if (dict_daily_dates_error_cal[each_month][day_ind] < weeks[week_ind + 1]):
                    week_list.append(weeks[week_ind])
                    days_list.append(days[day_ind])
                    month_list.append(month)
                    expiry_list.append(weeks[week_ind + 1])
                    day_ind = day_ind + 1
                elif (dict_daily_dates_error_cal[each_month][day_ind] == weeks[week_ind + 1]):
                    week_list.append(weeks[week_ind])
                    days_list.append(days[day_ind])
                    month_list.append(month)
                    expiry_list.append(weeks[week_ind + 1])
                    week_list.append(weeks[week_ind + 1])
                    days_list.append(days[day_ind])
                    month_list.append(month)
                    expiry_list.append(weeks[week_ind + 2])
                    day_ind = day_ind + 1
                    week_ind = week_ind + 1
            elif(week_ind < no_of_wks - 1):
                if (dict_daily_dates_error_cal[each_month][day_ind] < weeks[week_ind + 1]):
                    week_list.append(weeks[week_ind])
                    days_list.append(days[day_ind])
                    month_list.append(month)
                    expiry_list.append(weeks[week_ind + 1])
                    day_ind = day_ind + 1
                elif (dict_daily_dates_error_cal[each_month][day_ind] == weeks[week_ind + 1]):
                    week_list.append(weeks[week_ind])
                    days_list.append(days[day_ind])
                    month_list.append(month)
                    expiry_list.append(weeks[week_ind + 1])
                    week_list.append(weeks[week_ind + 1])
                    days_list.append(days[day_ind])
                    month_list.append(month)
                    expiry_list.append(month)
                    day_ind = day_ind + 1
                    week_ind = week_ind + 1
            else:
                if (dict_daily_dates_error_cal[each_month][day_ind] < month):
                    week_list.append(weeks[week_ind])
                    days_list.append(days[day_ind])
                    month_list.append(month)
                    expiry_list.append(month)
                    day_ind = day_ind + 1
                
                elif (dict_daily_dates_error_cal[each_month][day_ind] == month):
                    week_list.append(weeks[week_ind])
                    days_list.append(days[day_ind])
                    month_list.append(month)
                    expiry_list.append(month)
                    day_ind = day_ind + 1               
                                    
        days_list = [day.strftime("%d-%b-%Y") for day in days_list]
        week_list = [week.strftime("%d-%b-%Y") for week in week_list]
        expiry_list = [expiry.strftime("%d-%b-%Y") for expiry in expiry_list]
        month_list = [mth.strftime("%d-%b-%Y") for mth in month_list]
        
        full_days_list = full_days_list + days_list
        full_week_list = full_week_list + week_list
        full_month_list = full_month_list + month_list
        full_expiry_list = full_expiry_list + expiry_list
        
    day_wk = {"Week": full_week_list, "Day": full_days_list, "Month":full_month_list, "Expiry":full_expiry_list}
    df_wk_day_map = pd.DataFrame(day_wk)
    return df_wk_day_map


# Create dataframe which maps each day to the begining of the week
def create_df_wks(dict_wkly_expiries_each_month):
    week_dict ={}
    week_list = []
    for each_month in dict_wkly_expiries_each_month.keys():
        week_list = week_list + dict_wkly_expiries_each_month[each_month]
    week_list = [week.strftime("%d-%b-%Y") for week in week_list]
    week_dict = {"Week": week_list}
    df_wks = pd.DataFrame(week_dict)
    return df_wks



In [3]:
##################################
# Data Loading functions         #
#################################
 

#Load dataframe from csv file of the data for that month:
#Input:
#     Expiry dates list of options to be hedged in python date format
#     prod_types of the expiries = "CE", "PE", "FUT"
#Output:
#      dataframe with keys as same name as csv file
def load_all_mthly_data(monthly_expiries_list, input_path, holidays_list, prod_type_lists=["FUT", "CE", "PE"],
                        stock_ident = "BANKNIFTY"):
    df_dict = dict()
    for expiry_prod in list(itertools.product(monthly_expiries_list, prod_type_lists)):
        if (expiry_prod[0].month == 1):
                last_month = 12
                last_year = expiry_prod[0].year - 1
        else:
            last_month = expiry_prod[0].month - 1
            last_year = expiry_prod[0].year 
        
        start_date = prev_workday_if_holiday(find_last_thurs_date_of_month(last_year, \
                                                                                         last_month), 
                                                           holidays_list)

#         start_date = date_of_prev_thurs(expiry_prod[0], no_of_thurs_back=4)
        if (expiry_prod[1] == "FUT"):
            file_name = expiry_prod[1] + "IDX_" + stock_ident + "_" \
                        + start_date.strftime("%d-%b-%Y") +  "_TO_" \
                        + expiry_prod[0].strftime("%d-%b-%Y") + ".csv"
        else:
            file_name = "OPTIDX_" + stock_ident + "_" + expiry_prod[1] + "_" \
                        + start_date.strftime("%d-%b-%Y") +  "_TO_" \
                        + expiry_prod[0].strftime("%d-%b-%Y") + ".csv"
        df_dict["df" + "_" + file_name[:-4]] = pd.read_csv(input_path + file_name)
    return (df_dict)


def load_all_wkly_data_of_the_month(monthly_expiry, holidays_list, input_path, prod_type_lists=["CE", "PE"], 
                                    stock_ident = "BANKNIFTY"):
    df_dict = dict()
    
    if (monthly_expiry.month == 1):
        last_month = 12
        last_year = monthly_expiry.year - 1
    else:
        last_month = monthly_expiry.month - 1
        last_year = monthly_expiry.year 
        
    last_thurs_of_prev_month = prev_workday_if_holiday(find_last_thurs_date_of_month(last_year, \
                                                                                         last_month), 
                                                           holidays_list)
    wkly_expiries_list = []
    date_of_interest = last_thurs_of_prev_month
    while(date_of_interest < monthly_expiry):
        wkly_expiries_list.append(date_of_interest)
        date_of_interest = prev_workday_if_holiday(date_of_next_thurs(date_of_interest), holidays_list)
    
    for expiry_prod in list(itertools.product(wkly_expiries_list, prod_type_lists)):

        start_date = expiry_prod[0]
        end_date = prev_workday_if_holiday(date_of_next_thurs(expiry_prod[0]), holidays_list)

        file_name = "OPTIDX_" + stock_ident + "_" + expiry_prod[1] + "_" \
                        + start_date.strftime("%d-%b-%Y") +  "_TO_" \
                        + end_date.strftime("%d-%b-%Y") + ".csv"
        df_dict["df" + "_" + file_name[:-4]] = pd.read_csv(input_path + file_name)
        
    return (df_dict)


#Mibor Holidays dontmatch with NSE - 
#if mibor rate is missing, we take previous avaiable mibor rate
def find_latest_available_mibor(df_mibor_hist, curr_date):
    r_frame = df_mibor_hist[df_mibor_hist["Date"] == curr_date.strftime("%d-%b-%y")]["mibor_rate"]
    if (r_frame.empty):
        return find_latest_available_mibor(df_mibor_hist, curr_date - timedelta(days=1))
    else:
        return(r_frame.iloc[0])

In [4]:
###############################
# Dataframe Naming Functions #
##############################

# Funtion to get monthly data_frame names stored for each mkt_data from expiry_date
#Input: date, product type is "CE", "PE" or "FUT"
#Output: dataframe string name
def get_mthly_df_name_from_expiry(expiry_date, prod_type, holidays_list, stock_ident):
    
    if (expiry_date.month == 1):
        last_month = 12
        last_year = expiry_date.year - 1
    else:
        last_month = expiry_date.month - 1
        last_year = expiry_date.year 
        
    start_date = prev_workday_if_holiday(find_last_thurs_date_of_month(last_year, last_month), \
                                                       holidays_list)

#     start_date = date_of_prev_thurs(expiry_date, no_of_thurs_back=4)
#     start_date = prev_workday_if_holiday(start_date, holidays_list)
    if (prod_type == "FUT"):
        df_name_str = "df_" + prod_type + "IDX_" + stock_ident + "_" \
                    + start_date.strftime("%d-%b-%Y") +  "_TO_" \
                    + expiry_date.strftime("%d-%b-%Y")
    else:
            df_name_str = "df_" + "OPTIDX_" + stock_ident + "_" + prod_type + "_" \
                        + start_date.strftime("%d-%b-%Y") +  "_TO_" \
                        + expiry_date.strftime("%d-%b-%Y")      
    return df_name_str


In [12]:
#########################################################
# Function to Pull information on traded options in NSE #
#########################################################


#Function to find weekly short-term strikes
def generate_weekly_strikes(wkly_expiries_dict_each_month, relative_path, stock_ident = "BANKNIFTY"):
    dict_wkly_CE_strikes = {}
    dict_wkly_PE_strikes = {}
    dict_wkly_spots = {}
    for expiry_date_str in wkly_expiries_dict_each_month.keys():
        ce_strike_list = []
        pe_strike_list = []
        spot_list = []
        no_of_weeks = len(wkly_expiries_dict_each_month[expiry_date_str])
        for i in range(0, no_of_weeks):
            if (i < no_of_weeks-1):
                ce_filename = "OPTIDX_" + stock_ident + "_" + "CE" + "_" \
                            + wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y") +  "_TO_" \
                            + wkly_expiries_dict_each_month[expiry_date_str][i+1].strftime("%d-%b-%Y") + ".csv"
                pe_filename = "OPTIDX_" + stock_ident + "_" + "PE" + "_" \
                            + wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y") +  "_TO_" \
                            + wkly_expiries_dict_each_month[expiry_date_str][i+1].strftime("%d-%b-%Y") + ".csv"
            else:
                ce_filename = "OPTIDX_" + stock_ident + "_" + "CE" + "_" \
                            + wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y") +  "_TO_" \
                            + expiry_date_str + ".csv"
                pe_filename = "OPTIDX_" + stock_ident + "_" + "PE" + "_" \
                            + wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y") +  "_TO_" \
                            + expiry_date_str + ".csv"

            df_ce = pd.read_csv(relative_path + ce_filename)           
            df_ce_pre = df_ce[df_ce['Date'] == wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y")]
            
            if(df_ce_pre.empty):
                df_ce = df_ce[df_ce['Date'] == wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%y")]
            else:
                df_ce = df_ce_pre
            df_ce['Moneyness'] = df_ce['Underlying Value'] / df_ce['Strike Price']
            
            # Condition for getting short-term call strikes
            
            # Filter for liquidity
            trd_volume_array = np.sort(np.array(df_ce[df_ce['No. of contracts'] > 0]['No. of contracts']))
            trd_volume_50th_pct = np.percentile(trd_volume_array, 50)
            oi_array= np.sort(np.array(df_ce[df_ce['Open Int'] > 0]['Open Int']))
            oi_50th_pct = np.percentile(oi_array, 50)
            df_ce = df_ce[df_ce['Open Int'] > oi_50th_pct]
            df_ce = df_ce[df_ce['No. of contracts'] > trd_volume_50th_pct]
            # Filter for ATM and OTM
            df_ce = df_ce[df_ce['Moneyness'] <= 1.01]

            ce_strikes = df_ce['Strike Price'].tolist()
            
            df_pe = pd.read_csv(relative_path + pe_filename)
            df_pe_pre = df_pe[df_pe['Date'] == wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y")]
            
            if(df_pe_pre.empty):
                df_pe = df_pe[df_pe['Date'] == wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%y")]
            else:
                df_pe = df_pe_pre
            
            df_pe['Moneyness'] = df_pe['Strike Price'] / df_pe['Underlying Value'] 

            # Condition for getting short-term put strikes
            
            # Filter for liquidity            
            trd_volume_array = np.sort(np.array(df_pe[df_pe['No. of contracts'] > 0]['No. of contracts']))
            trd_volume_50th_pct = np.percentile(trd_volume_array, 50)
            oi_array= np.sort(np.array(df_pe[df_pe['Open Int'] > 0]['Open Int']))
            oi_50th_pct = np.percentile(oi_array, 50)
            df_pe = df_pe[df_pe['Open Int'] > oi_50th_pct]
            df_pe = df_pe[df_pe['No. of contracts'] > trd_volume_50th_pct]
            
            # Filter for ATM and OTM
            df_pe = df_pe[df_pe['Moneyness'] <= 1.01]

            pe_strikes = df_pe['Strike Price'].tolist()
            
            spot = df_ce['Underlying Value'].tolist()[0]
            ce_strike_list.append(ce_strikes)
            pe_strike_list.append(pe_strikes)
            spot_list.append(spot)
            

        dict_wkly_CE_strikes[expiry_date_str] = ce_strike_list
        dict_wkly_PE_strikes[expiry_date_str] = pe_strike_list
        dict_wkly_spots[expiry_date_str] = spot_list
    return dict_wkly_CE_strikes, dict_wkly_PE_strikes, dict_wkly_spots

def generate_weekly_CarrWu_strikes(wkly_expiries_dict_each_month, relative_path, stock_ident = "BANKNIFTY"):
    dict_wkly_CE_strikes = {}
    dict_wkly_PE_strikes = {}
    dict_wkly_spots = {}
    for expiry_date_str in wkly_expiries_dict_each_month.keys():
        ce_strike_list = []
        pe_strike_list = []
        spot_list = []
        no_of_weeks = len(wkly_expiries_dict_each_month[expiry_date_str])
        for i in range(0, no_of_weeks):
            if (i < no_of_weeks-1):
                ce_filename = "OPTIDX_" + stock_ident + "_" + "CE" + "_" \
                            + wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y") +  "_TO_" \
                            + wkly_expiries_dict_each_month[expiry_date_str][i+1].strftime("%d-%b-%Y") + ".csv"
                pe_filename = "OPTIDX_" + stock_ident + "_" + "PE" + "_" \
                            + wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y") +  "_TO_" \
                            + wkly_expiries_dict_each_month[expiry_date_str][i+1].strftime("%d-%b-%Y") + ".csv"
            else:
                ce_filename = "OPTIDX_" + stock_ident + "_" + "CE" + "_" \
                            + wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y") +  "_TO_" \
                            + expiry_date_str + ".csv"
                pe_filename = "OPTIDX_" + stock_ident + "_" + "PE" + "_" \
                            + wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y") +  "_TO_" \
                            + expiry_date_str + ".csv"

            df_ce = pd.read_csv(relative_path + ce_filename)           
            df_ce_pre = df_ce[df_ce['Date'] == wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y")]
            if(df_ce_pre.empty):
                df_ce = df_ce[df_ce['Date'] == wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%y")]
            else:
                df_ce = df_ce_pre
            
            df_ce['Moneyness'] = df_ce['Underlying Value'] / df_ce['Strike Price'] 

            # Condition for getting short-term call strikes
            
            # Filter for liquidity
            trd_volume_array = np.sort(np.array(df_ce[df_ce['No. of contracts'] > 0]['No. of contracts']))
            trd_volume_50th_pct = np.percentile(trd_volume_array, 50)
            oi_array= np.sort(np.array(df_ce[df_ce['Open Int'] > 0]['Open Int']))
            oi_50th_pct = np.percentile(oi_array, 50)
            df_ce = df_ce[df_ce['Open Int'] > oi_50th_pct]
            df_ce = df_ce[df_ce['No. of contracts'] > trd_volume_50th_pct]
            
            ce_strikes = df_ce['Strike Price'].tolist()
            
            df_pe = pd.read_csv(relative_path + pe_filename)
            df_pe_pre = df_pe[df_pe['Date'] == wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y")]
            if(df_pe_pre.empty):
                df_pe = df_pe[df_pe['Date'] == wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%y")]
            else:
                df_pe = df_pe_pre
            
            trd_volume_array = np.sort(np.array(df_pe[df_pe['No. of contracts'] > 0]['No. of contracts']))
            trd_volume_50th_pct = np.percentile(trd_volume_array, 50)
            oi_array= np.sort(np.array(df_pe[df_pe['Open Int'] > 0]['Open Int']))
            oi_50th_pct = np.percentile(oi_array, 50)
            df_pe = df_pe[df_pe['Open Int'] > oi_50th_pct]
            df_pe = df_pe[df_pe['No. of contracts'] > trd_volume_50th_pct]
            
            pe_strikes = df_pe['Strike Price'].tolist()
            
            spot = df_ce['Underlying Value'].tolist()[0]
            ce_strike_list.append(ce_strikes)
            pe_strike_list.append(pe_strikes)
            spot_list.append(spot)
            
        dict_wkly_CE_strikes[expiry_date_str] = ce_strike_list
        dict_wkly_PE_strikes[expiry_date_str] = pe_strike_list
        dict_wkly_spots[expiry_date_str] = spot_list
    return dict_wkly_CE_strikes, dict_wkly_PE_strikes, dict_wkly_spots



def generate_mthly_strikes(mthly_mkt_data, prod_moneyness, prod_type, holidays_list, stock_ident="BANKNIFTY"):
    dict_mthly_strikes = {}
    
    if (stock_ident == "BANKNIFTY"):
        for data in mthly_mkt_data.keys():
            if(data[3:6] != 'FUT' and data[20:22] == prod_type):
                if(len(mthly_mkt_data[data]['Expiry'][0].split("-")[2]) == 4):
                    date_format = "%d-%b-%Y"    
                else:
                    date_format = "%d-%b-%y"

                expiry_date = datetime.strptime(mthly_mkt_data[data]['Expiry'][0], date_format).date()
    #             expiry_date = datetime.strptime(mthly_mkt_data[data]['Expiry'][0], "%d-%b-%Y").date()

                if (expiry_date.month == 1):
                    last_month = 12
                    last_year = expiry_date.year - 1
                else:
                    last_month = expiry_date.month - 1
                    last_year = expiry_date.year 

                start_date = prev_workday_if_holiday(find_last_thurs_date_of_month(last_year, \
                                                                                   last_month), \
                                                     holidays_list)

                file_name = "df_OPTIDX_" + stock_ident + "_" + prod_type + "_" \
                                + start_date.strftime("%d-%b-%Y") +  "_TO_" \
                                + expiry_date.strftime("%d-%b-%Y") + ".csv"
                df_month = mthly_mkt_data[file_name[:-4]]
                df_month = df_month[df_month['Date'] == start_date.strftime(date_format)]

                # Filter for liquidity
                trd_volume_array = np.sort(np.array(df_month[df_month['No. of contracts'] > 0]['No. of contracts']))
                trd_volume_50th_pct = np.percentile(trd_volume_array, 50)
                oi_array= np.sort(np.array(df_month[df_month['Open Int'] > 0]['Open Int']))
                oi_50th_pct = np.percentile(oi_array, 50)
                df_month = df_month[df_month['Open Int'] > oi_50th_pct]
                df_month = df_month[df_month['No. of contracts'] > trd_volume_50th_pct]

                # Wht percentage to consider as OTM and ITM to be changed later
                if(prod_moneyness == 'ITM' and prod_type == "CE"):
                    df_month['sort_ind'] = np.abs(np.divide(df_month['Underlying Value'], df_month['Strike Price']) - 1.1)
                elif(prod_moneyness == 'OTM' and prod_type == "CE"):
                    df_month['sort_ind'] = np.abs(np.divide(df_month['Underlying Value'], df_month['Strike Price']) - 0.90)
                elif(prod_moneyness == 'ITM' and prod_type == "PE"):
                    df_month['sort_ind'] = np.abs(np.divide(df_month['Strike Price'], df_month['Underlying Value']) - 1.1)
                elif(prod_moneyness == 'OTM' and prod_type == "PE"):
                    df_month['sort_ind'] = np.abs(np.divide(df_month['Strike Price'], df_month['Underlying Value']) - 0.90)
                else:
                    df_month['sort_ind'] = np.abs(np.divide(df_month['Underlying Value'], df_month['Strike Price']) - 1)

                df_month = df_month.sort_values(by = ['sort_ind']) 
                dict_mthly_strikes[expiry_date.strftime("%d-%b-%Y")] = df_month['Strike Price'].iloc[0]
            
    else:
        for data in mthly_mkt_data.keys():
            if(data[3:6] != 'FUT' and data[16:18] == prod_type):
                if(len(mthly_mkt_data[data]['Expiry'][0].split("-")[2]) == 4):
                    date_format = "%d-%b-%Y"    
                else:
                    date_format = "%d-%b-%y"

                expiry_date = datetime.strptime(mthly_mkt_data[data]['Expiry'][0], date_format).date()
    #             expiry_date = datetime.strptime(mthly_mkt_data[data]['Expiry'][0], "%d-%b-%Y").date()

                if (expiry_date.month == 1):
                    last_month = 12
                    last_year = expiry_date.year - 1
                else:
                    last_month = expiry_date.month - 1
                    last_year = expiry_date.year 

                start_date = prev_workday_if_holiday(find_last_thurs_date_of_month(last_year, \
                                                                                   last_month), \
                                                     holidays_list)

                file_name = "df_OPTIDX_" + stock_ident + "_" + prod_type + "_" \
                                + start_date.strftime("%d-%b-%Y") +  "_TO_" \
                                + expiry_date.strftime("%d-%b-%Y") + ".csv"
                df_month = mthly_mkt_data[file_name[:-4]]
                df_month = df_month[df_month['Date'] == start_date.strftime(date_format)]

                # Filter for liquidity
                trd_volume_array = np.sort(np.array(df_month[df_month['No. of contracts'] > 0]['No. of contracts']))
                trd_volume_50th_pct = np.percentile(trd_volume_array, 50)
                oi_array= np.sort(np.array(df_month[df_month['Open Int'] > 0]['Open Int']))
                oi_50th_pct = np.percentile(oi_array, 50)
                df_month = df_month[df_month['Open Int'] > oi_50th_pct]
                df_month = df_month[df_month['No. of contracts'] > trd_volume_50th_pct]

                # Wht percentage to consider as OTM and ITM to be changed later
                if(prod_moneyness == 'ITM' and prod_type == "CE"):
                    df_month['sort_ind'] = np.abs(np.divide(df_month['Underlying Value'], df_month['Strike Price']) - 1.1)
                elif(prod_moneyness == 'OTM' and prod_type == "CE"):
                    df_month['sort_ind'] = np.abs(np.divide(df_month['Underlying Value'], df_month['Strike Price']) - 0.90)
                elif(prod_moneyness == 'ITM' and prod_type == "PE"):
                    df_month['sort_ind'] = np.abs(np.divide(df_month['Strike Price'], df_month['Underlying Value']) - 1.1)
                elif(prod_moneyness == 'OTM' and prod_type == "PE"):
                    df_month['sort_ind'] = np.abs(np.divide(df_month['Strike Price'], df_month['Underlying Value']) - 0.90)
                else:
                    df_month['sort_ind'] = np.abs(np.divide(df_month['Underlying Value'], df_month['Strike Price']) - 1)

                df_month = df_month.sort_values(by = ['sort_ind']) 
                dict_mthly_strikes[expiry_date.strftime("%d-%b-%Y")] = df_month['Strike Price'].iloc[0]

    return dict_mthly_strikes


# def generate_weekly_CarrWu_strikes(wkly_expiries_dict_each_month, relative_path, stock_ident = "BANKNIFTY"):
#     dict_wkly_CE_strikes = {}
#     dict_wkly_PE_strikes = {}
#     dict_wkly_spots = {}
#     for expiry_date_str in wkly_expiries_dict_each_month.keys():
#         ce_strike_list = []
#         pe_strike_list = []
#         spot_list = []
#         no_of_weeks = len(wkly_expiries_dict_each_month[expiry_date_str])
#         for i in range(0, no_of_weeks):
#             if (i < no_of_weeks-1):
#                 ce_filename = "OPTIDX_" + stock_ident + "_" + "CE" + "_" \
#                             + wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y") +  "_TO_" \
#                             + wkly_expiries_dict_each_month[expiry_date_str][i+1].strftime("%d-%b-%Y") + ".csv"
#                 pe_filename = "OPTIDX_" + stock_ident + "_" + "PE" + "_" \
#                             + wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y") +  "_TO_" \
#                             + wkly_expiries_dict_each_month[expiry_date_str][i+1].strftime("%d-%b-%Y") + ".csv"
#             else:
#                 ce_filename = "OPTIDX_" + stock_ident + "_" + "CE" + "_" \
#                             + wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y") +  "_TO_" \
#                             + expiry_date_str + ".csv"
#                 pe_filename = "OPTIDX_" + stock_ident + "_" + "PE" + "_" \
#                             + wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y") +  "_TO_" \
#                             + expiry_date_str + ".csv"

#             df_ce = pd.read_csv(relative_path + ce_filename)           
#             df_ce_pre = df_ce[df_ce['Date'] == wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y")]
#             if(df_ce_pre.empty):
#                 df_ce = df_ce[df_ce['Date'] == wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%y")]
#             else:
#                 df_ce = df_ce_pre
#             df_ce['Moneyness'] = df_ce['Underlying Value'] / df_ce['Strike Price'] 
#             df_ce = df_ce[df_ce['No. of contracts'] > 50]
#             df_ce = df_ce[df_ce['Open Int'] > 50] 
#             ce_strikes = df_ce['Strike Price'].tolist()
            
#             df_pe = pd.read_csv(relative_path + pe_filename)
#             df_pe_pre = df_pe[df_pe['Date'] == wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y")]
#             if(df_pe_pre.empty):
#                 df_pe = df_pe[df_pe['Date'] == wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%y")]
#             else:
#                 df_pe = df_pe_pre
#             df_pe['Moneyness'] = df_pe['Strike Price'] / df_pe['Underlying Value'] 
#             df_pe = df_pe[df_pe['No. of contracts'] > 50]
#             df_pe = df_pe[df_pe['Open Int'] > 50]
#             pe_strikes = df_pe['Strike Price'].tolist()
            
#             spot = df_ce['Underlying Value'].tolist()[0]
#             ce_strike_list.append(ce_strikes)
#             pe_strike_list.append(pe_strikes)
#             spot_list.append(spot)
            

#         dict_wkly_CE_strikes[expiry_date_str] = ce_strike_list
#         dict_wkly_PE_strikes[expiry_date_str] = pe_strike_list
#         dict_wkly_spots[expiry_date_str] = spot_list
#     return dict_wkly_CE_strikes, dict_wkly_PE_strikes, dict_wkly_spots



# def generate_mthly_strikes(mthly_mkt_data, prod_moneyness, prod_type, holidays_list, stock_ident="BANKNIFTY"):
#     dict_mthly_strikes = {}
#     for data in mthly_mkt_data.keys():
#         if(data[3:6] != 'FUT' and data[20:22] == prod_type):
#             if(len(mthly_mkt_data[data]['Expiry'][0].split("-")[2]) == 4):
#                 date_format = "%d-%b-%Y"    
#             else:
#                 date_format = "%d-%b-%y"
            
#             expiry_date = datetime.strptime(mthly_mkt_data[data]['Expiry'][0], date_format).date()
# #             expiry_date = datetime.strptime(mthly_mkt_data[data]['Expiry'][0], "%d-%b-%Y").date()
            
#             if (expiry_date.month == 1):
#                 last_month = 12
#                 last_year = expiry_date.year - 1
#             else:
#                 last_month = expiry_date.month - 1
#                 last_year = expiry_date.year 
        
#             start_date = prev_workday_if_holiday(find_last_thurs_date_of_month(last_year, \
#                                                                                last_month), \
#                                                  holidays_list)

#             file_name = "df_OPTIDX_" + stock_ident + "_" + prod_type + "_" \
#                             + start_date.strftime("%d-%b-%Y") +  "_TO_" \
#                             + expiry_date.strftime("%d-%b-%Y") + ".csv"
#             df_month = mthly_mkt_data[file_name[:-4]]
#             df_month = df_month[df_month['Date'] == start_date.strftime(date_format)]
            
#             df_month = df_month[df_month['No. of contracts'] > 50]
#             df_month = df_month[df_month['Open Int'] > 50] 
            
#             # Wht percentage to consider as OTM and ITM to be changed later
#             if(prod_moneyness == 'ITM' and prod_type == "CE"):
#                 df_month['sort_ind'] = np.abs(np.divide(df_month['Underlying Value'], df_month['Strike Price']) - 1.05)
#             elif(prod_moneyness == 'OTM' and prod_type == "CE"):
#                 df_month['sort_ind'] = np.abs(np.divide(df_month['Underlying Value'], df_month['Strike Price']) - 0.95)
#             elif(prod_moneyness == 'ITM' and prod_type == "PE"):
#                 df_month['sort_ind'] = np.abs(np.divide(df_month['Strike Price'], df_month['Underlying Value']) - 1.05)
#             elif(prod_moneyness == 'OTM' and prod_type == "PE"):
#                 df_month['sort_ind'] = np.abs(np.divide(df_month['Strike Price'], df_month['Underlying Value']) - 0.95)
#             else:
#                 df_month['sort_ind'] = np.abs(np.divide(df_month['Underlying Value'], df_month['Strike Price']) - 1)
            
#             df_month = df_month.sort_values(by = ['sort_ind']) 
#             dict_mthly_strikes[expiry_date.strftime("%d-%b-%Y")] = df_month['Strike Price'].iloc[0]
#     return dict_mthly_strikes

# The following function was replaced by the first function in this cell

# #Function to find weekly short-term strikes
# def generate_weekly_strikes(wkly_expiries_dict_each_month, relative_path, stock_ident = "BANKNIFTY"):
#     dict_wkly_CE_strikes = {}
#     dict_wkly_PE_strikes = {}
#     dict_wkly_spots = {}
#     for expiry_date_str in wkly_expiries_dict_each_month.keys():
#         ce_strike_list = []
#         pe_strike_list = []
#         spot_list = []
#         no_of_weeks = len(wkly_expiries_dict_each_month[expiry_date_str])
#         for i in range(0, no_of_weeks):
#             if (i < no_of_weeks-1):
#                 ce_filename = "OPTIDX_" + stock_ident + "_" + "CE" + "_" \
#                             + wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y") +  "_TO_" \
#                             + wkly_expiries_dict_each_month[expiry_date_str][i+1].strftime("%d-%b-%Y") + ".csv"
#                 pe_filename = "OPTIDX_" + stock_ident + "_" + "PE" + "_" \
#                             + wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y") +  "_TO_" \
#                             + wkly_expiries_dict_each_month[expiry_date_str][i+1].strftime("%d-%b-%Y") + ".csv"
#             else:
#                 ce_filename = "OPTIDX_" + stock_ident + "_" + "CE" + "_" \
#                             + wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y") +  "_TO_" \
#                             + expiry_date_str + ".csv"
#                 pe_filename = "OPTIDX_" + stock_ident + "_" + "PE" + "_" \
#                             + wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y") +  "_TO_" \
#                             + expiry_date_str + ".csv"

#             df_ce = pd.read_csv(relative_path + ce_filename)           
#             df_ce_pre = df_ce[df_ce['Date'] == wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y")]
#             if(df_ce_pre.empty):
#                 df_ce = df_ce[df_ce['Date'] == wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%y")]
#             else:
#                 df_ce = df_ce_pre
#             df_ce['Moneyness'] = df_ce['Underlying Value'] / df_ce['Strike Price']
#             df_ce = df_ce[df_ce['Moneyness'] <= 1.005]
#             df_ce = df_ce[df_ce['No. of contracts'] > 50]
#             df_ce = df_ce[df_ce['Open Int'] > 50] 
#             ce_strikes = df_ce['Strike Price'].tolist()
            
#             df_pe = pd.read_csv(relative_path + pe_filename)
#             df_pe_pre = df_pe[df_pe['Date'] == wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%Y")]
#             if(df_pe_pre.empty):
#                 df_pe = df_pe[df_pe['Date'] == wkly_expiries_dict_each_month[expiry_date_str][i].strftime("%d-%b-%y")]
#             else:
#                 df_pe = df_pe_pre
#             df_pe['Moneyness'] = df_pe['Strike Price'] / df_pe['Underlying Value'] 
#             df_pe = df_pe[df_pe['Moneyness'] <= 1.005]
#             df_pe = df_pe[df_pe['No. of contracts'] > 50]
#             df_pe = df_pe[df_pe['Open Int'] > 50]
#             pe_strikes = df_pe['Strike Price'].tolist()
            
#             spot = df_ce['Underlying Value'].tolist()[0]
#             ce_strike_list.append(ce_strikes)
#             pe_strike_list.append(pe_strikes)
#             spot_list.append(spot)
            

#         dict_wkly_CE_strikes[expiry_date_str] = ce_strike_list
#         dict_wkly_PE_strikes[expiry_date_str] = pe_strike_list
#         dict_wkly_spots[expiry_date_str] = spot_list
#     return dict_wkly_CE_strikes, dict_wkly_PE_strikes, dict_wkly_spots

In [13]:
###########################################################
# Processing of volatility surface generated from A1 code #
###########################################################

    
#Function for interpolation of iv - Linear Interpolation
#Other standard ways of interpolation could be tried in future
#This function can be used only for smile interpolation - not the entire surface
def gen_interpolated_iv(df_iv_surface, vol_date_str, moneyness):
    df_surf = df_iv_surface[df_iv_surface['Date']==vol_date_str]
    t_arr = np.sort(np.unique(np.array(df_surf['T'])))

    #Generated interpolated smile based on longest tenor options
    df_surf = df_surf[df_surf['T'] == t_arr[-1]]
    
    df_surf = df_surf.sort_values(by = ['Moneyness']) 
    k_by_s = np.array(df_surf['Moneyness'])
    imp_vol = np.array(df_surf['Impl_Vol'])
    if (moneyness <= k_by_s[0]):
        return imp_vol[0]
    elif (moneyness >= k_by_s[-1]):
        return imp_vol[-1]
    curr_moneyness = k_by_s[0]
    ind = 0
    while (moneyness >= curr_moneyness):
        ind = ind + 1
        curr_moneyness = k_by_s[ind]
    interpolated_value = imp_vol[ind-1] + ((imp_vol[ind] - imp_vol[ind-1]) / (k_by_s[ind] - k_by_s[ind-1])) * ((moneyness - k_by_s[ind-1]))
    return interpolated_value

#atm_ind = 1 for ATM else nothing
def generate_weekly_iv(dict_wkly_expiries_each_month, dict_wkly_spots, dict_mthly_strikes, \
                       prod_type, output_path, atm_ind, stock_ident):
    dict_wkly_iv = {}
    for each_mnth in dict_wkly_expiries_each_month.keys():
        no_of_weeks = len(dict_wkly_expiries_each_month[each_mnth])
        #Please note vol smile is same for European Call and Put
        df_daily_iv_data = pd.read_csv(output_path + "A1_" + stock_ident + "_" + "Daily_vol_smile_" + prod_type + "_" + each_mnth + ".csv")
        # Vol only above10^-5 - to avoid outliers in smile
        df_daily_iv_data = df_daily_iv_data[df_daily_iv_data['Impl_Vol'] > 0.00001]
        
        
        # The implied vol is taken based on the longest maturity tenor (i.e. of long options)
        # This functionality is taken care in the function gen_interpolated_iv
        if (prod_type == "PE"):
            df_iv = copy.deepcopy(df_daily_iv_data)
            # The following line is not required if directly call/put surface is used.
            # If common surface is used, this is required.
            df_iv['Moneyness'] = np.reciprocal(df_daily_iv_data['Moneyness'])
            
            iv_list = [gen_interpolated_iv(df_iv, (dict_wkly_expiries_each_month[each_mnth][i]).strftime("%d-%b-%Y"), ((dict_mthly_strikes[each_mnth]) / dict_wkly_spots[each_mnth][i])) for i in range(0, no_of_weeks)]

        else:
            df_iv = copy.deepcopy(df_daily_iv_data)
            iv_list = [gen_interpolated_iv(df_iv, (dict_wkly_expiries_each_month[each_mnth][i]).strftime("%d-%b-%Y"), (dict_wkly_spots[each_mnth][i] / (dict_mthly_strikes[each_mnth]))) for i in range(0, no_of_weeks)]
        
        dict_wkly_iv[each_mnth] = iv_list

    return dict_wkly_iv


#Function for interpolation of iv with respect to time to maturity and moneyness - Linear Interpolation
#Other standard ways of interpolation could be tried in future
def gen_bivariate_interpolated_iv(df_iv_surface, vol_date_str, moneyness, t_star):
    df_surf = df_iv_surface[df_iv_surface['Date']==vol_date_str]
    t_arr = np.sort(np.unique(np.array(df_surf['T'])))
#     print(t_star)
#     print(t_arr)
    if (t_star <= t_arr[0]):
        t_left = t_arr[0]
        t_right = t_arr[0]
    elif (t_star >= t_arr[-1]):
        t_left = t_arr[-1]
        t_right = t_arr[-1]
    else:
        time_ind = 0
        curr_time = t_arr[time_ind]
        while(t_star >= curr_time):
            time_ind = time_ind + 1
            curr_time = t_arr[time_ind]
        t_left = t_arr[time_ind-1]
        t_right = t_arr[time_ind]
    
    t_list = [t_left, t_right]
    
    impl_vol_list = []
    for time in t_list:
        df = df_surf[df_surf['T'] == time]
        df = df.sort_values(by = ['Moneyness'])
        k_by_s = np.array(df_surf['Moneyness'])
        imp_vol = np.array(df_surf['Impl_Vol'])
        if (moneyness <= k_by_s[0]):
            vol_left = imp_vol[0]
            vol_right = imp_vol[0]
            mny_left = k_by_s[0]
            mny_right = k_by_s[0]
        elif (moneyness >= k_by_s[-1]):
            vol_left = imp_vol[-1]
            vol_right = imp_vol[-1]
            mny_left = k_by_s[-1]
            mny_right = k_by_s[-1]
        else:
            curr_moneyness = k_by_s[0]
            ind = 0
            while (moneyness >= curr_moneyness):
                ind = ind + 1
                curr_moneyness = k_by_s[ind]
            vol_left = imp_vol[ind-1]
            vol_right = imp_vol[ind]
            mny_left = k_by_s[ind-1]
            mny_right = k_by_s[ind]
         
        if (mny_left == mny_right):
            inter_vol = vol_left
        else:
            inter_vol = vol_left + ((vol_right - vol_left) / (mny_right - mny_left)) * ((moneyness - mny_left))
        impl_vol_list.append(inter_vol)
    
    if (t_list[0] == t_list[1]):
        interpolated_value =  impl_vol_list[0]
    else:
        interpolated_value = impl_vol_list[0] + ((impl_vol_list[1] - impl_vol_list[0]) / (t_list[1] - t_list[0])) * (t_star - t_list[0])
        
    return interpolated_value

#Function for interpolation of iv with respect to time to maturity and moneyness - Linear Interpolation
#Other standard ways of interpolation could be tried in future
def gen_bivariate_forward_smile(df_iv_surface, vol_date_str, moneyness, t_star):
    df_surf = df_iv_surface[df_iv_surface['Date']==vol_date_str]
    t_arr = np.sort(np.unique(np.array(df_surf['T'])))
    
    if (t_star <= t_arr[0]):
        t_left = t_arr[0]
        t_right = t_arr[0]
    elif (t_star >= t_arr[-1]):
        t_left = t_arr[-1]
        t_right = t_arr[-1]
    else:
        time_ind = 0
        curr_time = t_arr[time_ind]
        while(t_star >= curr_time):
            time_ind = time_ind + 1
            curr_time = t_arr[time_ind]
        t_left = t_arr[time_ind-1]
        t_right = t_arr[time_ind]
    
    t_list = [t_left, t_right]
    
    impl_vol_list = []
    for time in t_list:
        df = df_surf[df_surf['T'] == time]
        df = df.sort_values(by = ['Moneyness'])
        k_by_s = np.array(df_surf['Moneyness'])
        imp_vol = np.array(df_surf['Impl_Vol'])
        if (moneyness <= k_by_s[0]):
            vol_left = imp_vol[0]
            vol_right = imp_vol[0]
            mny_left = k_by_s[0]
            mny_right = k_by_s[0]
        elif (moneyness >= k_by_s[-1]):
            vol_left = imp_vol[-1]
            vol_right = imp_vol[-1]
            mny_left = k_by_s[-1]
            mny_right = k_by_s[-1]
        else:
            curr_moneyness = k_by_s[0]
            ind = 0
            while (moneyness >= curr_moneyness):
                ind = ind + 1
                curr_moneyness = k_by_s[ind]
            vol_left = imp_vol[ind-1]
            vol_right = imp_vol[ind]
            mny_left = k_by_s[ind-1]
            mny_right = k_by_s[ind]
            
        if (mny_left == mny_right):
            inter_vol = vol_left
        else:
            inter_vol = vol_left + ((vol_right - vol_left) / (mny_right - mny_left)) * ((moneyness - mny_left))
        impl_vol_list.append(inter_vol)
            
    if (t_list[0] == t_list[1]):
        fwd_vol =  impl_vol_list[0]
    else:
        # Changed from vol to variance and taken square root
        fwd_vol = np.sqrt((((impl_vol_list[1]**2) * t_list[1]) - ((impl_vol_list[0]**2) * t_list[0])) / (t_list[1] - t_list[0]))
        
    return fwd_vol



In [1]:
################################################################################################
# Generate Interest rate curves - from Futures, MIBOR, Using Put-Call Parity or kept constant #
###############################################################################################

def generate_weekly_ir(dict_wkly_expiries_each_month, mthly_mkt_data, stock_ident):
    dict_wkly_ir = {}
    for each_month in dict_wkly_expiries_each_month.keys():
        month_date = datetime.strptime(each_month, "%d-%b-%Y").date()
        start_day = dict_wkly_expiries_each_month[each_month][0].strftime("%d-%b-%Y")
        df_fut = mthly_mkt_data['df_FUTIDX_' + stock_ident + '_' + start_day + '_TO_' + each_month]
        weekly_ir_list = []
        no_of_weeks = len(dict_wkly_expiries_each_month[each_month])
        for week in range(0, no_of_weeks):
            curr_day = dict_wkly_expiries_each_month[each_month][week]
            if(len(df_fut['Date'][0].split("-")[2]) == 4):
                df_week = df_fut[df_fut['Date'] == curr_day.strftime("%d-%b-%Y")]
            else:
                df_week = df_fut[df_fut['Date'] == curr_day.strftime("%d-%b-%y")]
            F_t = float(df_week['Settle Price'].tolist()[0])
#             F_t = float(df_week['Close'].tolist()[0])
            s_t = float(df_week['Underlying Value '].tolist()[0])
            disc_time = float((month_date - curr_day).days) / 365
            r = float(1 / disc_time) * np.log(F_t / s_t)
            weekly_ir_list.append(r)
        dict_wkly_ir[each_month] = weekly_ir_list
    return dict_wkly_ir

# Historical RBI repo rates
def get_risk_free_rate_from_exact_date(date_of_interest):
    if (date_of_interest < date(2019,8,7)):
        r_f = 0.0575
    elif (date_of_interest < date(2019,10,4)):
        r_f = 0.054
    elif (date_of_interest < date(2020,3,27)):
        r_f = 0.044
    else:
        r_f = 0.04
    return r_f


In [2]:
#####################
# Pricing Functions #
#####################

# r_f = 4.25%

#Black -Shcoles formula for European call options
def BSM_call(s_t, k, r, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log(s_t/k) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    return (s_t * norm.cdf(d_1) - k * np.exp(-r * dt) * norm.cdf(d_2))

def BSM_call_with_div(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log(s_t/k) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    return (np.exp(-q * dt) * s_t * norm.cdf(d_1) - k * np.exp(-r_f * dt) * norm.cdf(d_2))

def BSM_call_vec(s_t, k, r, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    return (np.multiply(norm.cdf(d_1),  s_t) - (norm.cdf(d_2) * k * np.exp(-r * dt)))


def BSM_call_vec_with_div(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    return (np.multiply(norm.cdf(d_1),  s_t) * np.exp(-q * dt) - (norm.cdf(d_2) * k * np.exp(-r_f * dt)))

#Black -Shcoles formula for European put options
def BSM_put(s_t, k, r, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log(s_t/k) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    return (k * np.exp(-r*dt) * norm.cdf(-d_2) - s_t *  norm.cdf(-d_1))

def BSM_put_with_div(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log(s_t/k) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r    
    return (k * np.exp(-r_f*dt) * norm.cdf(-d_2) -  np.exp(-q * dt) * s_t *  norm.cdf(-d_1))

def BSM_put_vec(s_t, k, r, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    return ((k * np.exp(-r*dt) * norm.cdf(-d_2)) - np.multiply(s_t, norm.cdf(-d_1)))

def BSM_put_vec_with_div(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r      
    return ((k * np.exp(-r_f*dt) * norm.cdf(-d_2)) - np.multiply(s_t, norm.cdf(-d_1)) * np.exp(-q * dt) )


In [16]:
#####################################
# Functions to generate simulations #
#####################################

#Function to generate covariance matrix
def generate_covariance_from_correlation(cor_mat, vol_list, dt):
    vol_diag_mat = np.diag(vol_list)
    cov_mat = np.dot(np.dot(vol_diag_mat, cor_mat), vol_diag_mat) * dt
    return cov_mat

#Generate Simulations of stocks
def gbm1W_simulation(no_of_paths, sim_start_date, sim_end_date, no_of_assets,
                     curr_stock_price, r, vol_list, cor_mat):
    zero_mean = np.zeros(no_of_assets)
    dt = float((sim_end_date - sim_start_date).days) / 365 
    cov_mat = generate_covariance_from_correlation(cor_mat, vol_list, dt)
    
    dw_matx = np.random.multivariate_normal(zero_mean, cov_mat, (int(no_of_paths / 2), 1))
    dw_mat = np.concatenate((dw_matx, -dw_matx), axis=0)
    dw_mat = dw_mat.reshape(no_of_paths, 1)

    sim_ln_stock_mat = np.zeros((no_of_paths, 2))
    sim_ln_stock_mat[:, 0] = np.log(curr_stock_price)
    base_drift = (r - 0.5 * np.square(vol_list[0])) * dt
    curr_drift = sim_ln_stock_mat[:, 0] + base_drift
    sim_ln_stock_mat[:, 1] = curr_drift + dw_mat[:, 0]
    sim_stock_mat = np.exp(sim_ln_stock_mat)
    
    return sim_stock_mat


In [17]:
###############################
# Functions related to Greeks #
##############################

# Function to find delta of call option with respect to futures price
def call_delta_with_futures(s_t, k, r, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    dc_ds = norm.cdf(d_1)
    df_ds = np.exp(r * dt)
    dc_df = float(dc_ds/df_ds)
    return dc_df


# Function to find delta of call option with respect to futures price
def call_delta_with_spot(s_t, k, r, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    dc_ds = norm.cdf(d_1)
    return dc_ds

def call_delta_with_spot_and_div(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    dc_ds = norm.cdf(d_1)
    q = r_f - r 
    return np.exp(- q * dt) * dc_ds

# Function to find delta of call option with respect to futures price
def put_delta_with_spot(s_t, k, r, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    dc_ds = norm.cdf(d_1)
    # put delta = call delta - 1
    return (dc_ds-1)

def put_delta_with_spot_and_div(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    q = r_f - r 
    dc_ds = np.exp(- q * dt) * norm.cdf(d_1)
    return (dc_ds-np.exp(-q*dt))

# Function to obtain gama with respect to spot
def gamma_with_spot(s_t, k, r, vol, dt):
    d_1_g = (1 / (vol * (dt ** 0.5)))
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_g * d1_log
    gamma = norm.pdf(d_1) * (1 / (vol * s_t * dt)) 
    return gamma


In [4]:
# monthly_expiry = date(2020,3,26) + timedelta(days=7)
# # holidays_list = [date(2020, 3, 10)]
# # wkly_expiries_list = [date_of_prev_thurs(monthly_expiry, no_of_thurs) for no_of_thurs in range(4, 0, -1)]
# # wkly_expiries_list = [prev_workday_if_holiday(date, holidays_list) for date in wkly_expiries_list]
# # print(wkly_expiries_list)
# print(monthly_expiry)

# 30255.75
# 29053.9
# 662.05

In [1]:
# print(call_delta_with_futures(30187, 30200, 0.0296546978301819, 0.19723714532458445, float(30/365)))

In [2]:
# call_delta_with_spot(30187, 30200, 0.0296546978301819, 0.19723714532458445, float(30/365))

In [3]:
# 0.525420513656548 * 30187

In [4]:
# 662.05 - 15860.869045750213

In [5]:
# -15198.819045750213 * np.exp(0.05/365)

In [6]:
# 0.525420513656548 * 29147.15

In [7]:
# 15314.510524624453 - 15200.901218367788

In [8]:
# 113.60930625666515-291.95

In [9]:
# np.array([1,2,34])[0]

In [10]:
# date(2020,1,30).month 

In [14]:
# periods_of_interest = {2019: [11, 12], 2020: [1,2,3]}
# holidays_list = [date(2020, 2, 21), date(2020, 3, 10)]
# expiries_of_interest = get_all_monthly_option_expiries(periods_of_interest, holidays_list)
# print(find_wkly_expries(expiries_of_interest, holidays_list))
# date_of_interest = date(2019,12,26)
# date_of_interest = prev_workday_if_holiday(date_of_next_thurs(date_of_interest), holidays_list)
# print(date_of_interest)

In [ ]:
#Black -Shcoles formula for European call options
# def BSM_call(s_t, k, r, vol, dt):
#     d_1_c = (1 / (vol * (dt ** 0.5))) 
#     d1_log =  (np.log(s_t/k) + ((r + (vol ** 2) / 2) * dt))
#     d_1 = d_1_c * d1_log
#     d_2 = d_1 - vol * (dt ** 0.5)
#     return (s_t * norm.cdf(d_1) - k * np.exp(-r * dt) * norm.cdf(d_2))



In [8]:
# def get_risk_free_rate_from_exact_date(date_of_interest):
#     if (date_of_interest < date(2019,8,7)):
#         r_f = 0.0575
#     elif (date_of_interest < date(2019,10,4)):
#         r_f = 0.054
#     elif (date_of_interest < date(2020,3,27)):
#         r_f = 0.044
#     else:
#         r_f = 0.04
#     return r_f


0.0575


In [5]:
# Repo rates
# 22 May 2020
# 4.00%

# 27 March 2020
# 4.40%

# 04 October, 2019
# 5.15%

# 07 August, 2019
# 5.40

# 06 June, 2019	5.75%

21

In [ ]:
# Historica Repo rate changes
# 22 May 2020	4.00%
# 27 March 2020	4.40%
# 04 October, 2019	5.15%
# 07 August, 2019	5.40%
# 06 June, 2019	5.75%
# 04 April, 2019	6%
# 07 February, 2019	6.25%
# 01 August, 2018	6.50%
# 06 June, 2018	6.25%
# 07 February, 2018	6.00%
# 02 August, 2017	6.00%
# 04 October, 2016	6.25%
# 05 April, 2016	6.50%
# 29 September, 2015	6.75%
# 02 June, 2015	7.25%
# 04 March, 2015	7.50%
# 15 January, 2015	7.75%
# 28 January, 2014	8.00%
# 29 October, 2013	7.75%
# 20 September, 2013	7.50%
# 03 May, 2013	7.25%
# 17 March, 2011	6.75%
# 25 January, 2011	6.50%
# 02 November, 2010	6.25%
# 16 September, 2010	6.00%
# 27 July, 2010	5.75%
# 02 July, 2010	5.50%
# 20 April, 2010	5.25%
# 19 March, 2010	5.00%
# 21 April, 2009	4.75%
# 05 March, 2009	5.00%
# 05 January, 2009	5.50%
# 08 December, 2008	6.50%
# 03 November, 2008	7.50%
# 20 October, 2008	8.00%
# 30 July, 2008	9.00%
# 25 June, 2008	8.50%
# 12 June, 2008	8.00%
# 30 March, 2007	7.75%
# 31 January, 2007	7.50%
# 30 October, 2006	7.25%
# 25 July, 2006	7.00%
# 24 January, 2006	6.50%
# 26 October, 2005	6.25%